In [4]:
#CELL 1
#TOKENIZING AND NORMALIZNG THE FULL TEXT COLUMN
import csv
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import bigrams
from collections import Counter

csv_file_path = 'news_stock_price.csv'

with open(csv_file_path, newline='') as csvfile:
    #Reading the file in
    myreader = csv.reader(csvfile)
    #Must put next command because for our csv file our headers were on the second row
    next(myreader)

    #Create an empty list for all of column 2(which was the full text column)
    ft_rows = []
    for row in myreader:
        ft_rows.append(row[2])

    row_count = 0
    for text in ft_rows:
        if row_count < 50:
            row_count += 1
    #Custom tokenization method to prevent valuable bigrams like S&P 500 from being removed
    def custom_tokenize(text):
        protected_text = text.replace('S&P 500', 'SNP500')
        tokens = nltk.word_tokenize(protected_text)
        restored_tokens = []
        for token in tokens:
            if token == 'SNP500':
                restored_tokens.extend(['S&P', '500'])
            else:
                restored_tokens.append(token)
        
        return restored_tokens

    combined_text = ' '.join(ft_rows)
    #creating stop words
    stop_words = stopwords.words('english')
    stop_words.extend(["said", "also", "like"])
    
    tokens = custom_tokenize(combined_text)
    
    normalized_tokens = [word for word in tokens if (word.isalpha() and word.lower() not in stop_words)or word in ['S&P', '500']]

In [17]:
#CELL 2
#FINDING OUT STATISTICS ABOUT OUR DATA
import csv
from nltk.tokenize import sent_tokenize

total_sentences = 0
total_words = 0
total_tokens = 0
unique_tokens = set()
unique_words = set()
count = 0
for text in ft_rows:
    #Total words
    total_words += len(text.split())
    #Total sentences using sentence tokenization
    sentences = sent_tokenize(text)
    total_sentences += len(sentences)
    
    #Total tokens
    mytokens = nltk.word_tokenize(text)
    total_tokens += len(mytokens)
 
average_sentences = total_sentences / len(ft_rows)
average_words = total_words / len(ft_rows)
average_tokens = total_tokens / len(ft_rows)
num_reports = len(ft_rows)

print("Total number of words in full text column: ", total_words)
print("Total number of tokens in full text column: ", total_tokens)
print("Total number of reports: ", num_reports)
print("Average number of sentences per report:", average_sentences)
print("Average number of words per report:", average_words)
print("Average number of tokens per report:", average_tokens)


Total number of words in full text column:  3468111
Total number of tokens in full text column:  4055377
Total number of reports:  4700
Average number of sentences per report: 33.72063829787234
Average number of words per report: 737.8959574468086
Average number of tokens per report: 862.846170212766


In [6]:
#CELL 3
#PRINTS OUT THE FIRST 500 tokens in the tokenized and normalized list
tcount = 0
tstring = ''
for token in normalized_tokens:
    if tcount < 500:
        tstring += token + ' '
        tcount += 1
print(tstring)

read account statements third quarter anything answer probably already know stocks bonds year date Looking losses make smaller might make us feel smaller natural avoid looking closely evidence might undermine belief skilled investors markets however making good decisions often requires admitting things would much rather ignore Let start recognizing inertia choice Since end March stocks within whisker highs investors withdrawn billion stock mutual funds funds according Investment Company Institute international stock funds held trillion end March even though inflation seem fall stocks seem rise investors taken money stock funds partly sheer inertia partly millions people invest autopilot partly changing course losing money feels intensely painful every investor recognizes wisdom old saying Cut losses let profits run thing worse losing admit loser investors avoid selling investment pretend paper loss work later hand realize loss without realizing made mistake Worse sold could go back wha

In [7]:
#CELL 4
#PRINTS THE TOP 50 WORDS OF NEW TOKENIZED AND NORMALIZED LIST
word_freq = nltk.FreqDist(normalized_tokens)
top_50_words = word_freq.most_common(50)

for word, freq in top_50_words:
    print(f"{word}: {freq}")


market: 24298
stock: 15747
stocks: 12283
investors: 10024
year: 9671
company: 7563
inflation: 7236
markets: 7033
500: 6966
S&P: 6532
per: 6218
growth: 6146
Fed: 5702
could: 5458
trading: 5449
would: 5427
time: 5309
companies: 5287
prices: 5195
week: 4924
last: 4845
earnings: 4794
since: 4756
higher: 4711
one: 4678
shares: 4672
new: 4638
rates: 4618
rate: 4586
years: 4581
points: 4565
billion: 4403
index: 4341
interest: 4332
price: 4321
percent: 4228
high: 4105
may: 3952
Nasdaq: 3647
data: 3564
first: 3525
economy: 3502
investment: 3333
share: 3296
even: 3253
quarter: 3236
well: 3171
financial: 3117
Stock: 3105
US: 3104


In [ ]:
#CELL 5
#Creating top 10 bigrams for the tokenized and normalized text
from nltk.util import bigrams

bigram_list = list(bigrams(normalized_tokens))

bigram_counts = Counter(bigram_list)
top_10_bigrams = bigram_counts.most_common(10)

print("Top 10 Bigrams:")
for bigram, count in top_10_bigrams:
    print(f"{bigram}: {count}")

In [ ]:
#CELL 6
#MAKING A WORD CLOUD FOR ALL TOKENS IN FULL_TEXT
#IM NOT SURE IF WE SHOULD LEMMATIZE OR NOT, EITHER WAY S&P doesnt show up in cloud
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

all_lemmas = [lemmatizer.lemmatize(w) for w in normalized_tokens]

text = " ".join(all_lemmas)

from wordcloud import WordCloud

word_freq = nltk.FreqDist(text)

top_words = dict(word_freq.most_common(100))
wordcloud = WordCloud(background_color='white').generate_from_frequencies(top_words)


import matplotlib.pyplot as plt
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis("off")
plt.show()


In [ ]:
#CELL 7
#FILTERS THE DATASET SO ONLY FULL TEXT REPORTS 
#WHERE A SPECIFIC WORD IS MENTIONED IN IT ARE INCLUDED. IT THEN ADDS THOSE REPORTS
#TO A NEW LIST(apple_rows) OF REPORTS THAT ALL INCLUDE THAT WORD 
#WE CAN USE THIS TO FIND BIGRAMS, DISTRIBUTIONS, and WORD CLOUDS FOR ALL REPORTS
#THAT MENTION THAT COMPANY. 

import csv
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import bigrams
from collections import Counter
import string
import spacy

csv_file_path = 'news_stock_price.csv'

column_name_to_search = 'full_text'

#SPECIFIC WORD TO MENTION. We can use this to find reports that mention a specific company
word_to_search = 'Tesla'

apple_rows = []

with open(csv_file_path, newline='') as csvfile:
    myreader = csv.reader(csvfile)

    next(myreader)
    headers = next(myreader)

    column_index = None

    column_index = headers.index(column_name_to_search)
    
    if column_index is not None:
        for row in myreader:
            if len(row) > column_index:
                if word_to_search in row[column_index]:
                    apple_rows.append(row[2])
apple_count = 0
for text in apple_rows:
    apple_count += text.count(word_to_search)
print(apple_count)


In [18]:
#CELL 8
#TOKENIZES AND NORMALIZES THE TEXT THAT MENTIONS THE COMPANY IN THE CELL ABOVE
#PRETTY MUCH THE SAME CODE AS I TOKENIZED BEFORE
#WE CAN ADD MORE WORDS/PHRASES TO CUSTOM TOKENIZE IF WE WANT TO PROTECT THEM (MAYBE % SIGN)
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import bigrams
from collections import Counter

def custom_tokenize(text):
    protected_text = text.replace('S&P 500', 'SNP500')
    tokens = nltk.word_tokenize(protected_text)

    restored_tokens = []
    for token in tokens:
        if token == 'SNP500':
            restored_tokens.extend(['S&P', '500'])
        else:
            restored_tokens.append(token)
    
    return restored_tokens

combined_text = ' '.join(apple_rows)

stop_words = stopwords.words('english')
stop_words.extend(["said", "also", "like"])

tokens = custom_tokenize(combined_text)
normalized_tokens = [word for word in tokens if (word.isalpha() and word.lower() not in stop_words) or word in ['S&P', '500']]
bigram_list = list(bigrams(normalized_tokens))

bigram_counts = Counter(bigram_list)
top_10_bigrams = bigram_counts.most_common(10)

print("Top 10 Bigrams:")
for bigram, count in top_10_bigrams:
    print(f"{bigram}: {count}")


NameError: name 'apple_rows' is not defined

In [ ]:
#CELL 9
#FINDING COLLOCATIONS
from nltk.collocations import *

def find_collocations(tokens, num):
    bigram_measures = BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(normalized_tokens)

    #Using pmi measures with a filter of 10. 
    finder.apply_freq_filter(10)
    top_n_bigrams = finder.nbest(bigram_measures.pmi, num)
    bigram_freqs = [(bigram, finder.ngram_fd[bigram]) for bigram in top_n_bigrams]
    return bigram_freqs
apple_collocations = find_collocations(normalized_tokens, 10)

print("Top 10 Collocations:")
for bigram, count in apple_collocations:
    print(f"{bigram}: {count}")

In [ ]:
#CEll 10
import csv  
from collections import defaultdict, Counter
import pandas as pd  
from datetime import datetime  # For handling date and time
import matplotlib.pyplot as plt  

# Defining a list of target words to track
target_words = ["Apple", "Facebook", "Microsoft", "Tesla", "Amazon"]

# Creating a defaultdict to store word frequencies over time
word_frequency_over_time = defaultdict(Counter)

with open('news_stock_price.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skipping the header row
    for row in reader:
        # Extracting news text and date from the row
        news_text = row[2]
        date = datetime.strptime(row[0], '%Y-%m-%d')  # Converting date string to datetime object

        # Splitting the news text into tokens (words)
        tokens = news_text.split()
        # Counting occurrences of target words in the news text
        word_counts = Counter(word for word in tokens if word in target_words)
        
        word_frequency_over_time[date].update(word_counts)

# Converting the defaultdict to a pandas DataFrame
df = pd.DataFrame.from_dict(word_frequency_over_time, orient='index')
# Filling missing values with 0
df = df.fillna(0)

# Plotting the data
df.plot(figsize=(12, 6))
plt.xlabel('Date')  
plt.ylabel('Word Frequency')  
plt.title('Word Frequency 
plt.legend(target_words)  
plt.xticks(rotation=45)  
plt.grid(True) 

# Saving the plot as an image file
plt.savefig('word_frequency_trends.png', dpi=300, bbox_inches='tight')
plt.show()




In [ ]:
# Importing necessary libraries
import spacy  # For recognizing entities using "en_core_web_lg" model from spaCy
import csv  
from collections import Counter  # For counting entity occurrences

# Load the large English model from Spacy
nlp = spacy.load("en_core_web_lg")

# Function to extract specified entities from text
def extract_entities(text):
    # Processing the text with the Spacy NLP model
    doc = nlp(text)
    # Returning entities that are organizations, geopolitical entities, or products
    return [ent.text for ent in doc.ents if ent.label_ in ["ORG", "GPE", "PRODUCT"]]

# Initializing a counter to keep track of entity occurrences
entity_counter = Counter()

# Open and read the CSV file
with open('news_stock_price.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  
    # Iterating over each row in the CSV file
    for row in reader:
        news_text = row[2]
        # Extracting entities from the news text
        entities = extract_entities(news_text)
        # Updating the counter with these entities
        entity_counter.update(entities)

# Printing the 10 most common entities
print("Most common entities:")
for entity, count in entity_counter.most_common(10):
    print(f"{entity}: {count}")

In [ ]:
import matplotlib.pyplot as plt

#draw plot using the ouput of last block, there is only eight since we combined "US and U.S., "NASDAQ and Nasdaq"
entities = ["U.S.", "Fed", "Nasdaq","Russia", "NYSE", "Ukraine", "China", "Treasury"]
frequencies = [5556 + 2520, 5403, 1495+1221, 2126, 2022, 1877, 1664, 1234]

plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
plt.bar(entities, frequencies, color='blue')
plt.xlabel('Entities')
plt.ylabel('Frequency')
plt.title('Most Common Entities Histogram (Combined U.S. and US)')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()
